In [1]:
from newsapi import NewsApiClient
import json
import datetime as dt
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os

In [2]:
newsapi = NewsApiClient(api_key=os.environ['NEWSAPI_KEY'])

In [3]:
#BUSINESS_SOURCES='bloomberg,cnbc,fortune,the-wall-street-journal'
# business-insider - not working well...

BUSINESS_SOURCES='bloomberg'

In [4]:
start_date = dt.datetime.today() - dt.timedelta(days=30)
end_date = dt.datetime.today()
news = newsapi.get_everything(q='Apple', sources=BUSINESS_SOURCES, from_param=str(start_date), to=str(end_date),
                              sort_by='publishedAt', page_size=100, page=1)

In [5]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://i.gyazo.com/ebcb9a03af1bd68283fcc0862d7195a5.png")

In [9]:
print('-- Sentiment analysis for Apple news\n')

analyzer = SentimentIntensityAnalyzer()

for article in news['articles']:
    print('Source: ' + article['source']['name'] + ' [' + article['publishedAt'] + ']')
    print('Title: ' + article['title'])
    if article['description']:
        print('"' + article['description'] + '"')

    text = article['description'] or article['title']
    print('> TextBlob : ' + str(TextBlob(text).sentiment))
    print('> VaderSentiment : ' + str(analyzer.polarity_scores(text)))

    print('')

-- Sentiment analysis for Apple news

Source: Bloomberg [2019-11-10T22:02:13Z]
Title: Apple Co-Founder Says Goldman's Apple Card Algo Discriminates
> TextBlob : Sentiment(polarity=0.0, subjectivity=0.0)
> VaderSentiment : {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Source: Bloomberg [2019-11-10T22:02:00Z]
Title: Goldman's Apple Card: Apple Co-Founder Wants Tougher Regulation - Bloomberg
"Goldman's Apple Card: Apple Co-Founder Wants Tougher Regulation Bloomberg New York regulator probes Apple Card algorithms for gender bias after viral tweets MarketWatch Apple's 'sexist' credit card investigated by US regulator BBC News New York regulator vows…"
> TextBlob : Sentiment(polarity=0.15757575757575756, subjectivity=0.33636363636363636)
> VaderSentiment : {'neg': 0.035, 'neu': 0.856, 'pos': 0.108, 'compound': 0.4404}

Source: Bloomberg [2019-11-09T20:52:41Z]
Title: Viral Tweet About Apple Card Leads to Probe Into Goldman Sachs
> TextBlob : Sentiment(polarity=0.0, subjectivity=0.0)
